In [1]:
import pandas as pd
import json
!pip install geopy
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
!pip install folium
import folium
import numpy as np
print("Libraries imported and installed")

     |████████████████████████████████| 111 kB 3.4 MB/s eta 0:00:01
     |████████████████████████████████| 93 kB 1.3 MB/s eta 0:00:011
Libraries imported and installed


In [2]:
data = [['Old_Town', 55.948726, -3.186815], 
        ['New_Town', 55.952972, -3.200384], 
        ['Leith', 55.975485, -3.170548], 
        ['Stockbridge', 55.958805, -3.210040], 
        ['Bruntsfield', 55.935706, -3.209900],
        ['Haymaket', 55.946128, -3.216803],
        ['Corstorphine', 55.942760, -3.282341],
        ['Portobello', 55.952860, -3.114287],
        ['South_Leith', 55.963663, -3.177896],
        ['Southside', 55.939108, -3.179524]]

df = pd.DataFrame(data, columns = ['Region', 'Latitude', 'Longitude']) 

df.style.set_caption("Selected Dinning Districts in Edinburgh")

,Region,Latitude,Longitude
0,Old_Town,55.9487,-3.18682
1,New_Town,55.953,-3.20038
2,Leith,55.9755,-3.17055
3,Stockbridge,55.9588,-3.21004
4,Bruntsfield,55.9357,-3.2099
5,Haymaket,55.9461,-3.2168
6,Corstorphine,55.9428,-3.28234
7,Portobello,55.9529,-3.11429
8,South_Leith,55.9637,-3.1779
9,Southside,55.9391,-3.17952


In [3]:
map_edinburgh = folium.Map(location=[55.951947, -3.193543], zoom_start=13)
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format('Popular Dinning Districts in Edinburgh')  
map_edinburgh.get_root().html.add_child(folium.Element(title_html))
for lat, lng, region in zip(df['Latitude'], df['Longitude'], df['Region']):
    folium.CircleMarker(
        [lat, lng],
        radius=30,
        popup=region,
        color='red',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_edinburgh)  
    
map_edinburgh


In [4]:
CLIENT_ID = '3N0SR5V4HMX20CMRYQEB2KWYEJTQAYBOL20DCVHP3TEMCF13' # your Foursquare ID
CLIENT_SECRET = 'LAR3Y5LPBH4SWJ5PBQY2K1E1MI2QZKQXZ2SRUEGNLSOVB3JU' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 1000 # A default Foursquare API limit value
CATEGORY_ID = '4d4b7105d754a06374d81259'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 3N0SR5V4HMX20CMRYQEB2KWYEJTQAYBOL20DCVHP3TEMCF13
CLIENT_SECRET:LAR3Y5LPBH4SWJ5PBQY2K1E1MI2QZKQXZ2SRUEGNLSOVB3JU


In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=700):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng,
            CATEGORY_ID,
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Region', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)


In [6]:
venues = getNearbyVenues(names=df['Region'],latitudes=df['Latitude'],longitudes=df['Longitude'])

Old_Town
New_Town
Leith
Stockbridge
Bruntsfield
Haymaket
Corstorphine
Portobello
South_Leith
Southside


In [7]:
print(venues.shape)
venues.head()

(468, 7)


,Region,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Old_Town,55.948726,-3.186815,The Edinburgh Larder,55.950080,-3.186088,Café
1,Old_Town,55.948726,-3.186815,Civerino's,55.949738,-3.188043,Pizza Place
2,Old_Town,55.948726,-3.186815,Mother India's Cafe,55.947998,-3.186139,Indian Restaurant
3,Old_Town,55.948726,-3.186815,The Holyrood 9A,55.949319,-3.182661,Gastropub
4,Old_Town,55.948726,-3.186815,Wings,55.949236,-3.189563,Wings Joint


In [8]:
total = venues.groupby('Region').count()[['Venue']].reset_index()
total.columns = ['Region','Total']
total = total.merge(df,on='Region')

map_edinburgh = folium.Map(location=[55.951947, -3.193543], zoom_start=13)
title_html = '''<h3 align="center" style="font-size:16px"><b>{}</b></h3>'''.format('Total number of restaurant in various districts')  
map_edinburgh.get_root().html.add_child(folium.Element(title_html))
for lat, lng, region, num in zip(total['Latitude'], total['Longitude'], total['Region'], total['Total'] ):
    label = folium.Popup(region, parse_html=True) 
    p1 = [lat,lng]
    folium.Marker(p1, icon=folium.features.DivIcon(
        icon_size=(150,36),
        icon_anchor=(15,15),
        html='<div style="font-size: 16pt; color : black">'+str(num)+'</div>',
        )).add_to(map_edinburgh)
    map_edinburgh.add_child(folium.CircleMarker(p1, radius=30))
map_edinburgh

In [9]:
total = venues.groupby('Region').count()[['Venue']].reset_index()
total.columns = ['Region','Total']
total = total.merge(df,on='Region')
total.style.set_caption("Total number of restaurants in different districts")

,Region,Total,Latitude,Longitude
0,Bruntsfield,31,55.9357,-3.2099
1,Corstorphine,9,55.9428,-3.28234
2,Haymaket,74,55.9461,-3.2168
3,Leith,29,55.9755,-3.17055
4,New_Town,100,55.953,-3.20038
5,Old_Town,100,55.9487,-3.18682
6,Portobello,11,55.9529,-3.11429
7,South_Leith,34,55.9637,-3.1779
8,Southside,37,55.9391,-3.17952
9,Stockbridge,43,55.9588,-3.21004
